In [31]:
import datetime

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

In [54]:
import os
from glob import glob

In [2]:
def get_masks(data): 
    train_mask = data['ForecastId'] == -1
    validation_mask = data['ForecastId'] == 0
    test_mask = data['ForecastId'] > 0
    return train_mask, validation_mask, test_mask

In [3]:
from sklearn.preprocessing import LabelEncoder
def add_time_features(data):
    le = LabelEncoder()
    data['Day_num'] = le.fit_transform(data.Date)
    data['Day'] = data['Date'].dt.day
    data['Month'] = data['Date'].dt.month
    return data

In [4]:
# from jupyterthemes import jtplot
# jtplot.style()

In [5]:
from tensorflow.keras.preprocessing import sequence

def extract_data(data, mask, features, targets, shift=1):
    df = data.loc[mask]
    X, y = df[features], df[targets]
    
    X = X.shift(shift).iloc[shift:]
    y = y.iloc[shift:]
    
    return X.to_numpy(), y.to_numpy()


def get_timeframes_dataset(X_train, y_train, window_size=4):
    x_res = []
    y_res = []
    for xs, ys in zip(X_train, y_train):
        data_gen = sequence.TimeseriesGenerator(xs, ys, window_size, batch_size=1)
        for x, y in data_gen:
            x_res.append(x[0])
            y_res.append(y[0])
            
    return np.array(x_res), np.array(y_res)


def generate_dataset(data, states, features, targets, window_size, train_mask, val_mask=None):
    X_train = []
    X_val = []
    y_train = []
    y_val = []
    for country in states.keys(): #tqdm(['Italy', 'China', 'US', 'Spain', 'Germany', 'Canada', 'Albania']):
        for state in states[country]:
            state_mask = (data.Country_Region == country) & (data.Province_State == state)
                            
            X, y = extract_data(data, train_mask & state_mask, features, targets)
            X_train.append(X)
            y_train.append(y)
            
            if val_mask is not None:
                X, y = extract_data(data, val_mask & state_mask, lstm_features, targets)
                X_val.append(X)
                y_val.append(y)
    
    X_train, y_train = get_timeframes_dataset(X_train, y_train, window_size)
    if val_mask is not None:
        X_val, y_val = get_timeframes_dataset(X_val, y_val, window_size)
        return X_train, y_train, X_val, y_val
    
    return X_train, y_train

#### TODO:
    - Add country specific features 

In [6]:
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

In [7]:
def create_model(n_features, n_targets, window_size, verbose=0):
    n_lstm_units = 10 * n_features
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(n_lstm_units, input_shape=(window_size, n_features), return_sequences=True),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.LSTM(n_lstm_units, return_sequences=True),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.05),

        tf.keras.layers.LSTM(n_lstm_units, return_sequences=True),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.LSTM(n_lstm_units),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(n_lstm_units, activation='relu'),
        tf.keras.layers.Dense(2 * n_features, activation='relu'),
        tf.keras.layers.Dense(n_targets, activation='relu'),
    ])
    if verbose > 0:
        model.summary()
    return model

In [8]:
import tensorflow.keras.backend as K

def rmsle(y, y0):
    return K.sqrt(K.mean(K.pow(K.log(y + 1) - K.log(y0 + 1), 2)))

In [10]:
def lr_schedule(epoch):
    lr = 1e-2
    if epoch > 10:
        lr = 1e-3
    if epoch > 20:
        lr = 5e-4
    if epoch > 70:
        lr = 1e-5
    print('Learning rate reduced: ', lr)
    return lr

In [11]:
def train_model(model, model_name, loss, metrics):
    model_name = model_name.replace(' ', '_')
    model.compile('adam', loss, metrics=metrics)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=0.0001, verbose=1)
    tnsboard = TensorBoard(log_dir=f'logs\\{model_name}')
    name_template = 'rmsle_{val_rmsle:.3f}_epoch_{epoch:02d}'
    checkout = ModelCheckpoint(f'weights\\{model_name}_{name_template}.hdf5', monitor=f'val_rmsle', save_best_only=True)
    callbacks = [tnsboard, reduce_lr, checkout]
    model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=128, epochs=400, callbacks=callbacks)
    return model

In [64]:
def predict_test(model, data, states, features, targets, window_size, train_mask, test_mask):
    submission_df = pd.DataFrame(data=data, columns=['ForecastId'] + targets, dtype=int)
    for country in tqdm(states.keys()):
        for state in states[country]:
            state_mask = (data.Country_Region == country) & (data.Province_State == state)

            pre_test_days = data.loc[train_mask & state_mask, features]
            X_test, _ = extract_data(data, test_mask & state_mask, features, targets, 0)

            X_test = np.concatenate([pre_test_days[-window_size:], X_test], axis=0)

            y_pred = np.zeros((len(X_test), len(targets)))
            for i in range(window_size, len(X_test)):
                test_window = np.expand_dims(X_test[i-window_size:i], axis=0)
                X_test[i, -2:] = model.predict(test_window)

            y_pred = X_test[window_size:, -2:]
            submission_df.loc[test_mask & state_mask, targets] = np.round(y_pred).astype(int)
    return submission_df[test_mask]

## Prepare data

In [45]:
train_data = pd.read_csv('data/in/train.csv').fillna('NaN').drop(columns=['Id'])
test_data = pd.read_csv('data/in/test.csv').fillna('NaN')

In [46]:
start_training = pd.to_datetime('2020-01-19')
train_up_to = pd.to_datetime('2020-03-25')
public_test_up_to = pd.to_datetime('2020-04-08')

In [47]:
train_data['Date'] = pd.to_datetime(train_data['Date'])
test_data['Date'] = pd.to_datetime(test_data['Date'])

public_mask = np.logical_and(start_training < train_data['Date'], train_data['Date'] <= train_up_to)
validation_mask = train_up_to < train_data['Date']

In [48]:
train_data.loc[public_mask, 'ForecastId'] = -1
train_data.loc[validation_mask, 'ForecastId'] = 0

test_data['ConfirmedCases'] = 0.0
test_data['Fatalities'] = 0.0

In [49]:
data = pd.concat([train_data, test_data], ignore_index=True)

In [50]:
states = {}
for c in data['Country_Region'].unique():
    states[c] = data[data['Country_Region'] == c]['Province_State'].unique()

In [51]:
train_mask, validation_mask, test_mask = get_masks(data)
data = add_time_features(data)

### Format data for LSTM

In [52]:
lstm_features = ['Day_num', 'Day', 'Month', 'ConfirmedCases', 'Fatalities']
lstm_targets = ['ConfirmedCases', 'Fatalities']
window_size = 4

train_mask, val_mask, test_mask = get_masks(data)
X_train, y_train, X_val, y_val = generate_dataset(data, states, lstm_features, lstm_targets, 
                                                  window_size, train_mask, val_mask)

In [53]:
n_features = len(lstm_features)
n_targets = len(lstm_targets)

model = create_model(n_features, n_targets, window_size, verbose=0)

In [55]:
def load_best_model(model_name, verbose=0):
    fname_template = os.path.join('weights', model_name + '*.hdf5')
    files = glob(fname_template)
    weights_fname = sorted(files)[0]
    model = tf.keras.models.load_model(weights_fname, custom_objects={'rmsle': rmsle})
    if verbose > 0:
        print(weights_fname)
        model.summary()
    return model

In [56]:
model_name = 'lstm_3xlstm_batchnorm_dropout_3xdense'
# train_model(model, model_name, 'msle', [rmsle])
model = load_best_model(model_name, verbose=1)

weights\lstm_3xlstm_batchnorm_dropout_3xdense_rmsle_0.337_epoch_397.hdf5
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 50)             11200     
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 50)             200       
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 50)             20200     
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 50)             200       
_________________________________________________________________
dropout (Dropout)            (None, 4, 50)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 50)             20200     
_________________________________________________

In [57]:
submission = predict_test(model, data, states, lstm_features, lstm_targets, window_size, train_mask, test_mask)

In [ ]:
#Plotting
# plt.figure(figsize=(14, 7))
# plt.plot(country_train_data['Date'], 
#          country_train_data['ConfirmedCases'].values, c='b', label='')

# plt.plot(country_val_data['Date'], 
#          country_val_data['ConfirmedCases'], c='b', label='Confirmed GT',)

# plt.scatter(country_train_data['Date'], y_pred_train['ConfirmedCases'], c='r', label='')
# plt.scatter(country_val_data['Date'], y_pred_val['ConfirmedCases'], c='r', label='Pred')

# plt.axvline(train_up_to, linestyle='--', label='Training Validation split')
# plt.legend()
# plt.title(country +' (Log Scale)')
# plt.ylabel('Log cases')
# plt.xlabel('Date')
# plt.show()